In [1]:
import pandas as pd
import numpy as np
import io
import os
import json
import distutils.dir_util
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import save_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense, SimpleRNN,Dropout

## 데이터 전처리

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# feature 추출용 data 준비
f_orig = pd.read_json('../kakao_arena_melon/arena_data/orig/train.json' ,typ = 'frame', encoding='utf-8')
df_org = pd.DataFrame(f_orig)
df_org = df_org.sort_values(by=['like_cnt'],ascending=False)
df_org = df_org[df_org['like_cnt']>5]

In [3]:
df_org.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53186 entries, 8699 to 31216
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          53186 non-null  object
 1   id            53186 non-null  int64 
 2   plylst_title  53186 non-null  object
 3   songs         53186 non-null  object
 4   like_cnt      53186 non-null  int64 
 5   updt_date     53186 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [4]:
# train X data 준비 (noise 포함된 버전)
f_train_X = pd.read_json('../kakao_arena_melon/arena_data/questions/train.json' ,typ = 'frame', encoding='utf-8')
df_train_X = pd.DataFrame(f_train_X)
df_train_X = df_train_X.sort_values(by=['like_cnt'],ascending=False)
df_train_X = df_train_X[df_train_X['like_cnt']>5]
df_train_X.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
21794,[],74220,HOT TRENDY POP: 놓쳐선 안될 'POP' (매주 업데이트),"[159317, 274905, 643023, 240703, 424328, 28087...",53211,2020-04-17 14:32:23.000
64038,"[감성, 때껄룩, 휴식, 떼껄룩, 팝]",122388,❤️때껄룩님 ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 플레이리스트❤️,"[432753, 330285, 354561, 12397, 559731, 406035...",41844,2020-04-23 20:58:45.000
23806,[],26174,듣다보면 '우와!' 하고 제목을 보게되는 팝,"[40160, 20039, 561488, 20051, 16242, 77028, 61...",27268,2020-04-19 21:22:11.000


In [5]:
df_train_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53186 entries, 21794 to 67019
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          53186 non-null  object
 1   id            53186 non-null  int64 
 2   plylst_title  53186 non-null  object
 3   songs         53186 non-null  object
 4   like_cnt      53186 non-null  int64 
 5   updt_date     53186 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [6]:
# train y data 준비 (noise 제거된 버전)
f_train_y = pd.read_json('../kakao_arena_melon/arena_data/answers/train.json' ,typ = 'frame', encoding='utf-8')
df_train_y = pd.DataFrame(f_train_y)
df_train_y = df_train_y.sort_values(by=['like_cnt'],ascending=False)
df_train_y = df_train_y[df_train_y['like_cnt']>5]
df_train_y.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
21794,"[노동요, 취향저격, 드라이브, Pop, 기분전환, 트렌디, 스트레스, 신나는]",74220,HOT TRENDY POP: 놓쳐선 안될 'POP' (매주 업데이트),"[368264, 600999, 452196, 380410, 388089, 25767...",53211,2020-04-17 14:32:23.000
64038,"[새벽, 잔잔한, 팝송, 힐링, 신나는]",122388,❤️때껄룩님 ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 플레이리스트❤️,"[532771, 383775, 176384, 152694, 60126, 142570...",41844,2020-04-23 20:58:45.000
23806,"[매장음악, 카페, 러블리쏘, 분위기, 드라이브, 여행, 매력적인, 기분전환, 팝]",26174,듣다보면 '우와!' 하고 제목을 보게되는 팝,"[256568, 380321, 20886, 144368, 175439, 118081...",27268,2020-04-19 21:22:11.000


In [7]:
df_train_y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53186 entries, 21794 to 67019
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          53186 non-null  object
 1   id            53186 non-null  int64 
 2   plylst_title  53186 non-null  object
 3   songs         53186 non-null  object
 4   like_cnt      53186 non-null  int64 
 5   updt_date     53186 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [8]:
# test data 준비 (noise 제거 버전)
f_test = pd.read_json('../kakao_arena_melon/arena_data/orig/val.json' ,typ = 'frame', encoding="utf-8")
df_test = pd.DataFrame(f_test)
df_test = df_test.sort_values(by=['like_cnt'],ascending=False)
df_test.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
16571,"[봄, 가을, 달달, 명곡, 힐링, 설렘, 사랑]",102123,여자들 취향저격하는 달달한 노래,"[481910, 216728, 460884, 547506, 541420, 10636...",20149,2019-11-08 05:31:14.000
20334,"[감성, 밤, 여름밤, 새벽감성, 새벽, 밤에듣기좋은노래, 새벽에듣기좋은노래, 잠들...",11762,불 꺼진 방 침대 속 자기 직전... 듣기 좋은 노래 ⭐,"[13815, 366786, 454528, 451593, 61159, 231078,...",19758,2020-04-17 03:16:32.000
2587,[팝],56212,내가 뽑은 최고의 팝송 TOP 100,"[672232, 430846, 264908, 588165, 19486, 668733...",18403,2012-03-19 10:49:17.000


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23015 entries, 16571 to 7385
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          23015 non-null  object
 1   id            23015 non-null  int64 
 2   plylst_title  23015 non-null  object
 3   songs         23015 non-null  object
 4   like_cnt      23015 non-null  int64 
 5   updt_date     23015 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [10]:
# 플레이리스트 내 song id 리스트
train_song = df_org['songs']
# 플레이리스트 내 tag 리스트
train_tag = df_org['tags']
# 플레이리스트 내 song id 리스트 전체 나열
song_list = [song for plist in train_song for song in plist]
# 플레이리스트 내 tag 리스트 전체 나열
tag_list = [tag for plist in train_tag for tag in plist]

In [11]:
# 전체 나열 리스트 중 각 song의 개수
count_song = Counter(song_list)
# 전체 나열 리스트 중 각 tag의 개수
count_tag = Counter(tag_list)

In [12]:
# 전체 플레이리스트 내 20회 넘게 담긴 song을 dictionary에 추가
mfl={}  # meaningful
for key, value in count_song.items():
    if value>20:
        mfl[key]=value
song_len = len(mfl) # 20회 이상 담긴 song의 개수
song_len

24537

In [13]:
# 전체 플레이리스트 내 10회 넘게 담긴 tag을 dictionary에 추가
for key, value in count_tag.items():
    if value>10:
        mfl[key]=value
tag_len=len(mfl)-song_len # 11회 이상 담긴 tag의 개수
tag_len

1619

In [14]:
mfl_col = list(mfl.keys())
len(mfl_col) # 유의미한 song과 tag의 총 개수 (song_len+tag_len)

26156

In [15]:
# # 전체 플레이리스트를 row, 유의미한 song과 tag를 column으로 하는 zero matrix 생성
# zero_matrix = np.zeros((len(df_train_X),len(mfl_col)))
# zero_matrix.shape

In [16]:
# df_zero = pd.DataFrame(zero_matrix,columns=mfl_col,index=df_train_X['id'])
# df_zero.head() # matrix >> dataframe 변환

In [17]:
# 각 플레이리스트에 해당하는 곡, 태그 좌표에 1 부여

def create_onehot(df,column_name):
    zero_matrix=np.zeros((len(df),len(column_name)))
    zero_df=pd.DataFrame(zero_matrix,columns=column_name,index=df['id'])
    for i in range(len(df)):
        for j in df.iloc[i,0]+df.iloc[i,3] :
            if j in column_name:
                zero_df.iloc[i,column_name.index(j)]=1
    return zero_df

In [18]:
# create_onehot 함수 활용 train X data >> onehot encoding
train_X_onehot = create_onehot(df_train_X, mfl_col)
train_X_onehot.shape

(53186, 26156)

In [19]:
# create_onehot 함수 활용 train y data >> onehot encoding
train_y_onehot = create_onehot(df_train_X, mfl_col)
train_y_onehot.shape

(53186, 26156)

In [24]:
# create_onehot 함수 활용 test data >> onehot encoding
test_onehot = create_onehot(df_test, mfl_col)
test_onehot.shape

(23015, 26156)

## 오토인코더(학습)

In [20]:
encoding_dim=64
input_pl = Input(shape=(len(mfl_col),)) # one-hot encoding된 DataFrame의 행이 Input
dropout = Dropout(0.2)(input_pl)
encoded = Dense(encoding_dim,activation='relu')(input_pl) # 64개로 축소(인코딩)
encoded = Dense(36,activation='relu')(encoded) # 36개로 축소(인코딩)

decoded = Dense(len(mfl_col),activation='sigmoid')(encoded) # 다시 원래 column 개수로 복원(디코딩)
autoencoder = Model(input_pl,decoded)

autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26156)]           0         
                                                                 
 dense (Dense)               (None, 64)                1674048   
                                                                 
 dense_1 (Dense)             (None, 36)                2340      
                                                                 
 dense_2 (Dense)             (None, 26156)             967772    
                                                                 
Total params: 2644160 (10.09 MB)
Trainable params: 2644160 (10.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
autoencoder.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
history = autoencoder.fit(train_X_onehot,train_y_onehot,epochs=10,batch_size=64,validation_split=0.2,verbose=1) # 기존의 데이터프레임 복원이 목적이기 때문에, test data == train data

Epoch 1/10
665/665 [==============================] - 83s 124ms/step - loss: 0.0670 - accuracy: 0.0086 - val_loss: 0.0044 - val_accuracy: 0.0101
Epoch 2/10
665/665 [==============================] - 85s 129ms/step - loss: 0.0049 - accuracy: 0.0095 - val_loss: 0.0037 - val_accuracy: 0.0101
Epoch 3/10
665/665 [==============================] - 101s 152ms/step - loss: 0.0043 - accuracy: 0.0095 - val_loss: 0.0035 - val_accuracy: 0.0101
Epoch 4/10
665/665 [==============================] - 92s 138ms/step - loss: 0.0040 - accuracy: 0.0098 - val_loss: 0.0033 - val_accuracy: 0.0094
Epoch 5/10
665/665 [==============================] - 101s 152ms/step - loss: 0.0038 - accuracy: 0.0091 - val_loss: 0.0032 - val_accuracy: 0.0089
Epoch 6/10
665/665 [==============================] - 81s 122ms/step - loss: 0.0037 - accuracy: 0.0099 - val_loss: 0.0031 - val_accuracy: 0.0121
Epoch 7/10
665/665 [==============================] - 93s 139ms/step - loss: 0.0035 - accuracy: 0.0142 - val_loss: 0.0030 - val_

In [23]:
autoencoder.save('autoencoder_denoise.h5')

C:\Users\SV\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# autoencoder = tf.keras.models.load_model('autoencoder_basic.h5')

In [25]:
predict_plist = autoencoder.predict(test_onehot)

720/720 [==============================] - 10s 13ms/step


In [26]:
df_id = list(df_test['id'])
col= mfl_col
ori_song = col[:song_len]
ori_tag = col[song_len:]

song_predict = predict_plist[:,:song_len] # song output(추천곡)
tag_predict = predict_plist[:,song_len:] # tag output(추천태그)

In [27]:
result=[]
n=0
for i in df_id:
    dic={}
    dic['id']=i

    plist_song=song_predict[n].argsort()[-100:] # predict한 song output 중 상위 100개
    p_song=[]
    for song in plist_song:
        p_song.append(ori_song[song])
    dic['songs']=p_song

    plist_tag=tag_predict[n].argsort()[-10:] # predict한 tag output 중 상위 10개
    p_tag=[]
    for tag in plist_tag:
        p_tag.append(ori_tag[tag])
    dic['tags']=p_tag
    n+=1
    result.append(dic)

In [28]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname+".json")
    distutils.dir_util.mkpath(parent)
    with io.open( fname+".json", "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [29]:
write_json(result,'results_denoise')